In [1]:
import os
from dotenv import load_dotenv 
load_dotenv("../../constants/.env")

True

In [2]:
api_key = os.environ.get('BINANCE_KEY')
secret_key = os.environ.get('BINANCE_SECRET')

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from pymongo import MongoClient

# Configuração do MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Substitua pela URI do seu banco de dados
db = client['trader_data']  # Nome do banco de dados
collection = db['opened_orders']  # Nome da coleção


In [4]:
import time
from binance.client import Client
from binance.enums import *

# Configurações da API
API_KEY = api_key
API_SECRET = secret_key
client = Client(API_KEY, API_SECRET)


In [5]:

# Parâmetros da estratégia
symbol = "VIDTUSDT"  # Ativo no mercado futuro
grid = 0.01  # Grid de 1% para abrir novas ordens
tp = 0.025  # Take Profit de 1%
sl = 0.004  # Stop Loss de 0.15%
leverage = 10  # Alavancagem no mercado futuro
amount_in_usd = 100  # Valor em USDT para cada ordem

# Configurar margem isolada, alavancagem e habilitar Hedge Mode na Binance
# client.futures_change_margin_type(symbol=symbol, marginType="ISOLATED")
client.futures_change_leverage(symbol=symbol, leverage=leverage)
# client.futures_change_position_mode(dualSidePosition=True)  # Habilita Hedge Mode


{'symbol': 'VIDTUSDT', 'leverage': 10, 'maxNotionalValue': '800000'}

In [6]:

# Funções auxiliares
def adjust_to_tick_size(price, tick_size):
    """Ajusta o preço para ser múltiplo do tick size."""
    return round(price // tick_size * tick_size, len(str(tick_size).split('.')[-1]))

def get_symbol_info(symbol):
    """Obtém informações do símbolo, incluindo tick size."""
    info = client.futures_exchange_info()
    for s in info['symbols']:
        if s['symbol'] == symbol:
            for f in s['filters']:
                if f['filterType'] == 'PRICE_FILTER':
                    return float(f['tickSize'])
    raise ValueError(f"Informações do símbolo {symbol} não encontradas.")

def get_price():
    """Obtém o preço atual do ativo no mercado futuro."""
    ticker = client.futures_symbol_ticker(symbol=symbol)
    return float(ticker["price"])

def get_quantity(price, amount_in_usd):
    """Calcula a quantidade de contratos com base no valor em USD."""
    return round(amount_in_usd / price, 0)  # Binance aceita até 6 casas decimais para quantidade

def get_level(price, grid_base):
    """Calcula o nível discreto baseado no grid."""
    return int(price // grid_base)

def open_position(side, price, level, last_level):
    """
    Abre uma posição no mercado futuro com SL e TP configurados.
    
    Parâmetros:
        side (str): 'BUY' para long ou 'SELL' para short.
        price (float): Preço no momento da abertura.
    """
    tick_size = get_symbol_info(symbol)  # Obter o tick size do ativo
    quantity = get_quantity(price, amount_in_usd)
    
    if side == "BUY":
        tp_price = adjust_to_tick_size(price * (1 + tp), tick_size)
        sl_price = adjust_to_tick_size(price * (1 - sl), tick_size)
        position_side = "LONG"
    elif side == "SELL":
        tp_price = adjust_to_tick_size(price * (1 - tp), tick_size)
        sl_price = adjust_to_tick_size(price * (1 + sl), tick_size)
        position_side = "SHORT"
    
    # Abrir posição de mercado
    order = client.futures_create_order(
        symbol=symbol,
        side=side,
        type=ORDER_TYPE_MARKET,
        quantity=quantity,
        positionSide=position_side  # Especificar posição para o modo Hedge
    )
    
    # Configurar SL como uma ordem STOP_MARKET
    if (side == "BUY" and sl_price < price) or (side == "SELL" and sl_price > price):
        client.futures_create_order(
            symbol=symbol,
            side="SELL" if side == "BUY" else "BUY",
            type="STOP_MARKET",
            quantity=quantity,
            stopPrice=sl_price,
            positionSide=position_side  # Garantir que o SL seja compatível com a posição
        )
    else:
        print(f"SL inválido: SL {sl_price} já atingido ou muito próximo do preço atual {price}")
    
    # Configurar TP como uma ordem LIMIT
    if (side == "BUY" and tp_price > price) or (side == "SELL" and tp_price < price):
        client.futures_create_order(
            symbol=symbol,
            side="SELL" if side == "BUY" else "BUY",
            type=ORDER_TYPE_LIMIT,
            quantity=quantity,
            price=tp_price,
            timeInForce=TIME_IN_FORCE_GTC,
            positionSide=position_side  # Garantir que o TP seja compatível com a posição
        )
    else:
        print(f"TP inválido: TP {tp_price} já atingido ou muito próximo do preço atual {price}")
    
    print(f"Posição {side} aberta em {price} com TP: {tp_price} e SL: {sl_price}")

    df_wallet = pd.DataFrame(client.futures_account()["assets"])
    walletBalance = df_wallet[df_wallet['asset'] == 'USDT']['walletBalance'].values[0]

    data = {
        "side":side,
        "price":price,
        "tp_price":tp_price,
        "sl_price":sl_price,
        "level": level,
        "last_level": last_level,
        "walletBalance":walletBalance,
        "tp":tp,
        "sl": sl,
        "grid":grid,
        "symbol":symbol,
        "leverage":leverage,
        "amount_in_usd": amount_in_usd
    }
        
    collection.insert_one(data)


In [7]:
# price = get_price()
# print(price)
# get_quantity(price, amount_in_usd)

In [8]:
# Fluxo principal
initial_price = get_price()  # Obter preço inicial
grid_base = initial_price * grid  # Base do grid em termos absolutos
last_level = 0
"""Executa o algoritmo de grid trading."""
while True:
    try:
        
        price = get_price()
        level = get_level(price, grid_base)
        
        # Verifica se é necessário abrir uma posição long
        if level != last_level:
            df_opened_orders = pd.DataFrame(client.futures_get_open_orders())
            if len(df_opened_orders) == 0:  
                open_position("BUY", price, level, last_level)
                open_position("SELL", price, level, last_level)

                last_level = level 
            
        # Aguarda antes de verificar novamente
        time.sleep(2)  # 2 segundos entre verificações
    except Exception as e:
        print(f"Erro: {e}")
        time.sleep(2)  # Em caso de erro, aguardar antes de tentar novamente




Posição BUY aberta em 0.03044 com TP: 0.0312 e SL: 0.03031
Posição SELL aberta em 0.03044 com TP: 0.02967 e SL: 0.03056
Posição BUY aberta em 0.03043 com TP: 0.03119 e SL: 0.0303
Posição SELL aberta em 0.03043 com TP: 0.02966 e SL: 0.03055
Posição BUY aberta em 0.0308 com TP: 0.03156 e SL: 0.03067
Posição SELL aberta em 0.0308 com TP: 0.03002 e SL: 0.03092
Posição BUY aberta em 0.03174 com TP: 0.03253 e SL: 0.03161
Posição SELL aberta em 0.03174 com TP: 0.03094 e SL: 0.03186
Posição BUY aberta em 0.03227 com TP: 0.03307 e SL: 0.03214
Posição SELL aberta em 0.03227 com TP: 0.03146 e SL: 0.03239


KeyboardInterrupt: 

In [ ]:
# Posição BUY aberta em 0.03018 com TP: 0.03048 e SL: 0.03002
# Posição SELL aberta em 0.03018 com TP: 0.02987 e SL: 0.03033
# Posição BUY aberta em 0.02986 com TP: 0.03015 e SL: 0.02971
# Posição SELL aberta em 0.02986 com TP: 0.02956 e SL: 0.03
# Posição BUY aberta em 0.03034 com TP: 0.03064 e SL: 0.03018
# Posição SELL aberta em 0.03034 com TP: 0.03003 e SL: 0.03049
# Posição BUY aberta em 0.03079 com TP: 0.03109 e SL: 0.03063
# Posição SELL aberta em 0.03079 com TP: 0.03048 e SL: 0.03094
# Posição BUY aberta em 0.03124 com TP: 0.03155 e SL: 0.03108
# Posição SELL aberta em 0.03124 com TP: 0.03092 e SL: 0.03139
# Posição BUY aberta em 0.0317 com TP: 0.03201 e SL: 0.03154
# Posição SELL aberta em 0.0317 com TP: 0.03138 e SL: 0.03185
# Posição BUY aberta em 0.03215 com TP: 0.03247 e SL: 0.03198
# Posição SELL aberta em 0.03215 com TP: 0.03182 e SL: 0.03231

In [21]:
def get_level(price, grid_base):
    """Calcula o nível discreto baseado no grid."""
    return int(price // grid_base)

In [49]:
price = 0.02977
grid_base = price * grid  # Base do grid em termos absolutos
get_level(price, grid_base)

66

In [54]:
price = 0.0294
get_level(price, grid_base)

65

In [55]:
long_level

67

In [56]:
short_level

67

In [57]:
(0.02977-0.0294)/0.02977

0.012428619415519055